## Imports

In [ ]:
!pip3 install -U git+https://github.com/PYFTS/pyFTS

  Cloning https://github.com/PYFTS/pyFTS to /tmp/pip-req-build-cv38yacj
  Running command git clone --filter=blob:none --quiet https://github.com/PYFTS/pyFTS /tmp/pip-req-build-cv38yacj
  Resolved https://github.com/PYFTS/pyFTS to commit 4e0ee7cdd9fc9a8ac9170c0c3496536ef0378cb7
  Preparing metadata (setup.py) ... done
  Created wheel for pyFTS: filename=pyFTS-1.7-py3-none-any.whl size=219432 sha256=9b7bf863e0ac2b7cc08cda88f9d8c0b240c68d2df9c96bb272e2202af219898d
  Stored in directory: /tmp/pip-ephem-wheel-cache-2kpt5t46/wheels/7f/25/75/dd8f713bf48592da71e0441c7abdc1f5aa4d9645a6b7ee99d4
Successfully built pyFTS


In [ ]:
import numpy as np
import os
import seaborn as sns
import pandas as pd
from math import e,sqrt
from pyFTS.fcm import Activations
from pyFTS.models import hofts
from numpy.linalg import svd
import matplotlib.pyplot as plt
from pyFTS.fcm import fts as fcm_fts
from pyFTS.partitioners import Grid,CMeans
from pyFTS.common import Util
from pyFTS.common import Membership as mf

from scipy.optimize import least_squares
from scipy.optimize import leastsq
from pyFTS.benchmarks import Measures
from pyFTS.common import Util
from sklearn.decomposition import PCA, KernelPCA
from sklearn.preprocessing import StandardScaler, KernelCenterer, MinMaxScaler
from sklearn.utils import extmath
from sklearn.metrics.pairwise import euclidean_distances
import datetime
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score

/usr/local/lib/python3.10/dist-packages/pyFTS/common/transformations/trend.py:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  from pandas import datetime


## Model

### Common codes

In [ ]:

import numpy as np
from math import e,sqrt

def step(x, deriv=False):
    if deriv:
        return np.abs(1 * (x == 0))
    else:
        return np.abs(1 * (x > 0))


def sigmoid(x, deriv=False):
    if deriv:
        #return sigmoid(x)*(1 - sigmoid(x))
        return x * (1 - x)
    else:
        return 1 / (1 + np.exp(-x))


def softmax(x, deriv=False):
    if deriv:
        pass
    else:
        mvs = sum([np.exp(k) for k in x.flatten()])
        return np.array([np.exp(k)/mvs for k in x.flatten()])


def tanh(x, deriv=False):
    if deriv:
        return 1 - np.tanh(x)**2
    else:
        return np.tanh(x)


def relu(x, deriv=False):
    if deriv:
        return np.abs(1. * (x > 0))
    else:
        return np.abs(x * (x > 0))


# SoftPlus
def softplus(x,deriv=False):
    if deriv:
        return 1/(1+((e)**-x))
    else:
        return np.log(1+((e)**x))

# Bent identity:
def bentid(x,deriv=False):
    if deriv:
        return (x/(2*(sqrt((x**2)+1))))+1
    else:
        return (sqrt(((x**2)+1)-1)/2)+x

# SoftExponential:
def softex(x,a=0.01,deriv=False):
    if deriv:
        return 1/(1-a*(a+x)) if a < 0 else (e)**(a*x)
    else:
        if a < 0:
            return -((np.log(1-a*(x+a)))/a)
        elif a == 0:
            return x
        elif a > 0:
            return (((e)**(a*x))/a)+a


# Gaussian:
def gaussian(x,deriv=False):
    if deriv:
        return -2*x*(e)**((-x)**2)
    else:
        return (e)**((-x)**2)




### EmbeddingPCA

In [ ]:
#Embedding PCA and KPCA
class EmbeddingPCA():
    def __init__(self, no_of_components,gamma):
        self.no_of_components = no_of_components
        self.eigen_values = None
        self.eigen_vectors = None

        # debug attributes
        self.name = 'Principle Component Analysis'
        self.shortname = 'PCA'

        self.std = StandardScaler()
        self.pca_sk = PCA(n_components=self.no_of_components)
        self.gamma = gamma
        self.kpca_sk = KernelPCA(n_components=self.no_of_components, kernel='rbf', gamma=gamma, fit_inverse_transform = True)

    def __str__(self):
        return self.name

    def transform(self, x):
        return np.dot(x - self.mean, self.projection_matrix.T)

    def inverse_transform(self, x):
        return np.dot(x, self.projection_matrix) + self.mean

    def fit(self, x):
        self.no_of_components = x.shape[1] if self.no_of_components is None else self.no_of_components
        self.mean = np.mean(x, axis=0)

        cov_matrix = np.cov(x - self.mean, rowvar=False)

        self.eigen_values, self.eigen_vectors = np.linalg.eig(cov_matrix)
        self.eigen_vectors = self.eigen_vectors.T

        self.sorted_components = np.argsort(self.eigen_values)[::-1]

        self.projection_matrix = self.eigen_vectors[self.sorted_components[:self.no_of_components]]
        self.explained_variance = self.eigen_values[self.sorted_components];
        self.explained_variance_ratio = self.explained_variance / self.eigen_values.sum()

    def pca_sklearn(self,x):
        return self.pca_sk.fit_transform(x);

    def pca_sklearn_inverse(self, x):
        return self.pca_sk.inverse_transform(x);

    def standardization(self, x):
        return self.std.fit_transform(x)

    def standardization_inverse(self,x):
        return self.std.inverse_transform(x)

    def kernel_pca(self,x, gamma):
        # Calculate euclidean distances of each pair of points in the data set
        dist = euclidean_distances(x, x, squared=True)

        # Calculate Kernel matrix
        k = np.exp(-gamma * dist)
        kc = KernelCenterer().fit_transform(k)

        # Get eigenvalues and eigenvectors of the kernel matrix
        eig_vals, eig_vecs = np.linalg.eigh(kc)

        # flip eigenvectors' sign to enforce deterministic output
        eig_vecs, _ = extmath.svd_flip(eig_vecs, np.empty_like(eig_vecs).T)

        # Concatenate the eigenvectors corresponding to the highest n_components eigenvalues
        xkpca = np.column_stack([eig_vecs[:, -i] for i in range(1, self.no_of_components + 1)])

        return xkpca

    def kernel_pca_sklearn(self, x, gamma):
        #self.kpca_sk = KernelPCA(n_components=self.no_of_components, kernel='rbf', gamma=gamma)
        return self.kpca_sk.fit_transform(x)

    def kernel_pca_sklearn_inverse(self, x, gamma):
        return self.kpca_sk.inverse_transform(x)

### FCM

In [ ]:

class FuzzyCognitiveMap(object):
    def __init__(self, **kwargs):
        super(FuzzyCognitiveMap, self).__init__()
        self.order = kwargs.get('order',1)
        self.concepts = kwargs.get('partitioner',None)
        self.weights = []
        self.bias=[]
        self.activation_function = kwargs.get('activation_function', Activations.sigmoid)
        #self.activation_function=softplus

    def activate(self, memberships):
        dot_products = np.zeros(len(self.concepts))
        for k in np.arange(0, self.order):
            dot_products += np.dot(np.array(memberships[k]).T, self.weights[k])+(self.bias[k])
        return self.activation_function( dot_products )


### Loss Functions

In [ ]:
import numpy as np

def func(values:np.ndarray, *arg:np.float64):
    #print(values.shape)
    if len(values.shape) == 1:
        n_coefs = values.shape[0] + 1
      #return arg[0] + np.sum([values[i-1] * arg[i] for i in np.arange(1,n_coefs)])
        ret = 0
    else:
        n_coefs = values.shape[0] + 1
        n_inst = values.shape[1]
        ret = np.zeros(n_inst)
    for k in range(1, n_coefs):
        ret = ret + arg[k] * np.array (values[k-1])

    ret = ret + arg[0]

    return ret


def func2(x, a, b, c):
    z1,z2 = x
    return a + (b * z1) + (c * z2)

def func3(x, a, b, c, d):
    z1,z2,z3 = x
    return a + (b * z1) + (c * z2) + (d * z3)

def func4(x, a, b, c, d, e):
    z1,z2,z3,z4 = x
    return a + (b * z1) + (c * z2) + (d * z3) + (e * z4)

### MRHFCM

In [ ]:
class Deep_Random_HFCM(hofts.HighOrderFTS):

    def __init__(self, **kwargs):
        super(Deep_Random_HFCM, self).__init__(**kwargs)

        num_concepts:int = self.partitioner.partitions

        self.num_fcms:int = kwargs.get('num_fcms', 2)

        self.fcm:list = []

        self.loss_function = kwargs.get('loss', func)

        for k in range(self.num_fcms):
            fcm_tmp = FuzzyCognitiveMap(**kwargs)
            weights:np.ndarray = np.random.uniform(-1, 1, size=(self.order,num_concepts, num_concepts))
            specturalradius1:np.float=np.max(np.abs(np.linalg.eigvals(weights)))
            fcm_tmp.weights:np.ndarray = weights*0.5/specturalradius1
            bias:np.ndarray = np.random.uniform(-1, 1, size=(self.order,num_concepts))
            U,S,VT=svd(bias)
            specturalradius2:np.float=np.max(S)
            fcm_tmp.bias:np.ndarray=bias*0.5/specturalradius2
            self.fcm.append(fcm_tmp)


        # Coefficients

        #self.theta = np.zeros(self.num_fcms + 1)
        self.theta = np.random.rand(self.num_fcms + 1)

    def layers(self,fcm: FuzzyCognitiveMap,data:np.ndarray,**kwargs)->list:
        forecasts=[]
        midpoints = np.array([fset.centroid for fset in self.partitioner])
        for t in np.arange(self.order,len(data)+1):
            sample = data[t - self.order : t]
            fuzzyfied = self.partitioner.fuzzyfy(sample, mode='vector')
            activation = fcm.activate(fuzzyfied)
            forecasts.append(np.dot(midpoints, activation)/np.nanmax([1, np.sum(activation)]))
        return forecasts

    def forecast(self, data:np.ndarray, **kwargs)->list:
        y1 = []

        all_forecasts = []
        for l,fcm in enumerate(self.fcm):
            if l == 0:
                forecasts = self.layers(fcm,data)

                forecasts=np.pad(forecasts, (self.order-1), 'constant',constant_values=(0))
                forecasts=forecasts[:len(forecasts)-(self.order-1)]
                #print('shape data',np.shape(data))
                all_forecasts.append(forecasts)
                #print('forecasts1 shape',np.shape(forecasts))
                #print('all_forecasts1 shape',np.shape(all_forecasts))
            else:

                forecasts = self.layers(fcm,forecasts)
                forecasts=np.pad(forecasts, (self.order-1), 'constant',constant_values=(0))
                forecasts=forecasts[:len(forecasts)-(self.order-1)]
                all_forecasts.append(forecasts)
                #print('all_forecasts2',np.shape(all_forecasts))
                #print('forecasts2',np.shape(forecasts))



        result = self.loss_function(np.array(all_forecasts), *self.theta)
#         if str(result) == 'nan' or result == np.nan or result == np.Inf:
#             print('error')

        y1.append(result)

        return y1

    def run_fcm(self, fcm, data):
        ret = []
        midpoints = np.array([fset.centroid for fset in self.partitioner])
        for t in np.arange(self.order, len(data)+1):
            sample = data[t - self.order : t]
            fuzzyfied = self.partitioner.fuzzyfy(sample, mode='vector')
            activation = fcm.activate(fuzzyfied)
            final = np.dot(midpoints, activation)/np.nanmax([1, np.sum(activation)])
            ret.append(final)
        return ret

    def train(self, data:np.ndarray, **kwargs):
        from scipy.optimize import curve_fit, least_squares, minimize, leastsq

        outputs = []


        for model in self.fcm:
            outputs.append(self.run_fcm(model, data)[:-1])

        f = lambda coef, y, x: self.loss_function(x, *coef) - y

          #xdata = tuple(outputs)
      #print('shape of y',y.shape)
          #self.theta, cov = curve_fit(self.loss_function, xdata=outputs, ydata=data[self.order:], p0=self.theta )
                                        #n=self.num_fcms+1)#,
        self.theta, flag = leastsq(f, x0 = self.theta, args=(data[self.order:], np.array(outputs)))
      #print(self.theta)

### MRHFCM_PCA

In [ ]:
import numpy as np
import pandas as pd

from pyFTS.models.nonstationary import partitioners as nspart
from pyFTS.models.nonstationary import nsfts
from pyFTS.fcm import Activations

class DRHFCM_PCA():
    def __init__(self, num_components_pca, gamma):
        # debug attributes
        self.name = 'Deep Random High Order FCM-PCA '
        self.shortname = 'DRHFCM_PCA'

        ## Parameters of the model
        self.num_components_pca = num_components_pca

        self.gamma = gamma


        # PCA
        self.pca = EmbeddingPCA(no_of_components=self.num_components_pca, gamma = self.gamma)

    def run_test(self, models, data,transformation):
        if transformation == 'PCA':
            pca_test = self.create_pca_components(data)
        else:
            pca_test = self.create_kernel_pca_components(data)

        components = pca_test.columns
        forecast_model = []

        for i in range(0, self.num_components_pca):
            mf = models[i].predict(np.array(pca_test.loc[:,components[i]]))

            mf = np.array(mf)
            mf = mf.reshape(mf.shape[1],1)

            if (i == 0):
                forecast_model = np.array(mf)
            else:
                forecast_model = np.column_stack((forecast_model,mf))

        if transformation == 'PCA':
            forecast = self.inverse_transformation(forecast_model)
        else:
            forecast = self.inverse_kpca_transformation(forecast_model)

        return forecast, data

    def run_train(self, data,transformation):
        if transformation == 'PCA':
            pca_train = self.create_pca_components(data)
        else:
            pca_train = self.create_kernel_pca_components(data)

        models = self.create_and_fit_mimo_models(pca_train)

        return models,pca_train

    def create_and_fit_mimo_models(self, data):
        components = data.columns
        models = []

        for i in range(0, self.num_components_pca):
            md = self.create_DRHFCM_FTS(data.loc[:,components[i]])
            md.fit(np.array(data.loc[:,components[i]]))
            models.append(md)

        return models

    def create_DRHFCM_FTS(self,data):
        partitioner = Grid.GridPartitioner(data=data, npart=3, mf=mf.trimf)

        model = Deep_Random_HFCM(partitioner=partitioner, order=3, num_fcms=2,
                  activation_function=relu,
                  loss=func)


        return model

    def create_pca_components(self,data):
        transformed = self.pca.standardization(data)
        x_std = self.pca.pca_sklearn(transformed)

        components = []
        for i in range(0, self.num_components_pca):
            components.append('C' + str(i))

        df_pca = pd.DataFrame(x_std, columns=list(components))
        return df_pca

    def create_names_compoments(self):
        components = None
        for i in range (0,self.num_components_pca):
            components.append()

    def create_kernel_pca_components(self,data):
        transformed = self.pca.standardization(data)
        x_std = self.pca.kernel_pca_sklearn(transformed,self.gamma)

        components = []
        for i in range(0, self.num_components_pca):
            components.append('C' + str(i))

        df_kpca = pd.DataFrame(x_std, columns=list(components))
        return df_kpca

    def inverse_transformation(self, data):
        inverse_pca = self.pca.pca_sklearn_inverse(data)
        final_result = self.pca.standardization_inverse(inverse_pca)
        return final_result

    def inverse_kpca_transformation(self, data):
        inverse_kpca = self.pca.kernel_pca_sklearn_inverse(data,self.gamma)
        final_result = self.pca.standardization_inverse(inverse_kpca)
        return final_result

    def prepare_data(self,data):
        df = pd.DataFrame(data,columns=['component'])
        return df['component'].values

## Dataset

In [ ]:
import numpy as np
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame)
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

In [ ]:
# convert series to supervised learning
def series_to_supervised_mimo(data, n_in, n_out, dropnan=True):
	"""
	Frame a time series as a supervised learning dataset.
	Arguments:
		data: Sequence of observations as a list or NumPy array.
		n_in: Number of lag observations as input (X).
		n_out: Number of observations as output (y).
		dropnan: Boolean whether or not to drop rows with NaN values.
	Returns:
		Pandas DataFrame of series framed for supervised learning.
	"""
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [(df.columns[j]+'(t-%d)' % (i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [(df.columns[j]+'(t)') for j in range(n_vars)]
		else:
			names += [(df.columns[j]+'%d(t+%d)' % (j, i)) for j in range(n_vars)]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [ ]:

import pandas as pd

# OMID's LOCAL DATASET FILE
#df = pd.read_csv('C:\\Users\BAHAR\Desktop\RANDOM HFCM\multivariate\energydata_complete.csv', sep=',')

df = pd.read_csv('/content/energydata_complete.csv', sep=',')
df = df.drop(labels=['date','rv1', 'rv2'], axis=1)
df = clean_dataset(df)
df.head()


<ipython-input-11-1e44e683937a>:5: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)


,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,...,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint
0,60.0,30.0,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,...,18.2,48.900000,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3
1,60.0,30.0,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,...,18.2,48.863333,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2
2,50.0,30.0,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,...,18.2,48.730000,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1
3,50.0,40.0,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,...,18.1,48.590000,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0
4,60.0,40.0,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,...,18.1,48.590000,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9


In [ ]:
round(df.describe(),3)

,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,...,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint
count,19735.000,19735.000,19735.000,19735.000,19735.000,19735.000,19735.000,19735.000,19735.000,19735.000,...,19735.000,19735.000,19735.000,19735.000,19735.000,19735.000,19735.000,19735.000,19735.000,19735.000
mean,97.695,3.802,21.687,40.260,20.341,40.420,22.268,39.243,20.855,39.027,...,22.029,42.936,19.486,41.552,7.412,755.523,79.750,4.040,38.331,3.761
std,102.525,7.936,1.606,3.979,2.193,4.070,2.006,3.255,2.043,4.341,...,1.956,5.224,2.015,4.151,5.317,7.399,14.901,2.451,11.795,4.195
min,10.000,0.000,16.790,27.023,16.100,20.463,17.200,28.767,15.100,27.660,...,16.307,29.600,14.890,29.167,-5.000,729.300,24.000,0.000,1.000,-6.600
25%,50.000,0.000,20.760,37.333,18.790,37.900,20.790,36.900,19.530,35.530,...,20.790,39.067,18.000,38.500,3.667,750.933,70.333,2.000,29.000,0.900
50%,60.000,0.000,21.600,39.657,20.000,40.500,22.100,38.530,20.667,38.400,...,22.100,42.375,19.390,40.900,6.917,756.100,83.667,3.667,40.000,3.433
75%,100.000,0.000,22.600,43.067,21.500,43.260,23.290,41.760,22.100,42.157,...,23.390,46.536,20.600,44.338,10.408,760.933,91.667,5.500,40.000,6.567
max,1080.000,70.000,26.260,63.360,29.857,56.027,29.236,50.163,26.200,51.090,...,27.230,58.780,24.500,53.327,26.100,772.300,100.000,14.000,66.000,15.500


In [ ]:
reframed = series_to_supervised_mimo(df, 1, 1)
reframed.head()

,Appliances(t-1),lights(t-1),T1(t-1),RH_1(t-1),T2(t-1),RH_2(t-1),T3(t-1),RH_3(t-1),T4(t-1),RH_4(t-1),...,T8(t),RH_8(t),T9(t),RH_9(t),T_out(t),Press_mm_hg(t),RH_out(t),Windspeed(t),Visibility(t),Tdewpoint(t)
1,60.0,30.0,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,...,18.2,48.863333,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2
2,60.0,30.0,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,...,18.2,48.730000,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1
3,50.0,30.0,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,...,18.1,48.590000,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0
4,50.0,40.0,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,...,18.1,48.590000,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9
5,60.0,40.0,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,...,18.1,48.590000,17.000000,45.29,6.016667,734.0,92.0,5.333333,43.833333,4.8


In [ ]:
# def sample_first_prows(data, perc):
#     return data.head(int(len(data)*(perc)))

In [ ]:
#df = df.head(15000)
#train = sample_first_prows(df,0.75)
#test = df.iloc[max(train.index):]
#train_values = train.values
#test_values = test.values

In [ ]:
#models, data_train = Deep_random_HFCM.run_train(train_values,'PCA')
#forecast, data_test = Deep_random_HFCM.run_test(models,test_values,'PCA')

## Experiments

### Sliding Window Cross Validation

In [ ]:
import math
def cal_nrmse(rmse, y):
    x = max(y)-min(y)
    return (rmse/x)
def sliding_window_pca(data,n_windows,train_size,transformation):

    result = {
         "window": [],
         "rmse": [],
         "mape": [],
         "mae": [],
         "r2": [],
         "nrmse": [],
         "variable":[]
    }

    final_result = {
         "window": [],
         "rmse": [],
         "mape": [],
         "mae": [],
         "r2": [],
         "nrmse": [],
         "variable":[]
    }

    tam = len(data)
    n_windows = 30
    windows_length = math.floor(tam / n_windows)
    for ct, ttrain, ttest in Util.sliding_window(data, windows_length, train_size, inc=1):
        if len(ttest) > 0:

            print('-' * 20)
            print(f'training window {(ct)}')


            df_train = ttrain.loc[:,'Appliances(t-1)':'Tdewpoint(t-1)']
            df_test = ttest.loc[:,'Appliances(t-1)':'Tdewpoint(t-1)']
            df_original = ttest.loc[:,'Appliances(t)':'Tdewpoint(t)']

            models, data_train = Deep_random_HFCM.run_train(df_train,transformation)
            forecast, data_test = Deep_random_HFCM.run_test(models,df_test,transformation)


            df_forecats_columns = ['Appliances(t)', 'lights(t)', 'T1(t)', 'RH_1(t)', 'T2(t)', 'RH_2(t)',
                                   'T3(t)', 'RH_3(t)', 'T4(t)', 'RH_4(t)', 'T5(t)', 'RH_5(t)', 'T6(t)',
                                   'RH_6(t)', 'T7(t)', 'RH_7(t)', 'T8(t)', 'RH_8(t)', 'T9(t)', 'RH_9(t)',
                                   'T_out(t)', 'Press_mm_hg(t)', 'RH_out(t)', 'Windspeed(t)',
                                   'Visibility(t)', 'Tdewpoint(t)']
            columns = list(df_forecats_columns)
            df_forecast = pd.DataFrame(forecast,columns=columns)


            for col in columns:
                m=Deep_random_HFCM.create_DRHFCM_FTS(df_test)
                order=m.order
                print('order',order)
                num_fcms=m.num_fcms
                print('num_fcms',num_fcms)
                original = df_original[col].values
#
                forecast = df_forecast[col].values
#

                print("[{0: %H:%M:%S}]".format(datetime.datetime.now()) + f" getting statistics for variable: " + col)

                mae = round(mean_absolute_error(original,forecast),3)
                r2 = round(r2_score(original,forecast),3)
                #rmse = mean_squared_error(original,forecast,squared=False)
                rmse = round(Measures.rmse(original,forecast),3)
                mape = round(Measures.mape(original,forecast),3)
                nrmse = round(cal_nrmse(rmse, original),3)

                result["rmse"].append(rmse)
                result["nrmse"].append(nrmse)
                result["mape"].append(mape)
                result["mae"].append(mae)
                result["r2"].append(r2)
                result["window"].append(ct)
                result["variable"].append(col)
    measures = pd.DataFrame(result)
    return measures



In [ ]:
Deep_random_HFCM=DRHFCM_PCA(num_components_pca =2,gamma = 0.9)

In [ ]:
#df = df.head(50000)
data = reframed
for experiment in range(0,1):
    print('n_experiment is:', experiment)
    kpca_result =  sliding_window_pca(data=data,n_windows=30,train_size=0.75,transformation='kPCA')


n_experiment is: 0
--------------------
training window 0
shape data (164,)
forecasts1 shape (164,)
all_forecasts1 shape (1, 164)
all_forecasts2 (2, 164)
forecasts2 (164,)
order 3
num_fcms 2
[ 05:33:18] getting statistics for variable: Appliances(t)
order 3
num_fcms 2
[ 05:33:18] getting statistics for variable: lights(t)
order 3
num_fcms 2
[ 05:33:18] getting statistics for variable: T1(t)
order 3
num_fcms 2
[ 05:33:18] getting statistics for variable: RH_1(t)
order 3
num_fcms 2
[ 05:33:18] getting statistics for variable: T2(t)
order 3
num_fcms 2
[ 05:33:18] getting statistics for variable: RH_2(t)
order 3
num_fcms 2
[ 05:33:18] getting statistics for variable: T3(t)
order 3
num_fcms 2
[ 05:33:18] getting statistics for variable: RH_3(t)
order 3
num_fcms 2
[ 05:33:18] getting statistics for variable: T4(t)
order 3
num_fcms 2
[ 05:33:18] getting statistics for variable: RH_4(t)
order 3
num_fcms 2
[ 05:33:18] getting statistics for variable: T5(t)
order 3
num_fcms 2
[ 05:33:18] getting

C:\Users\BAHAR\anaconda3\lib\site-packages\pyFTS\benchmarks\Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
C:\Users\BAHAR\anaconda3\lib\site-packages\pyFTS\benchmarks\Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100


 3
num_fcms 2
[ 05:33:18] getting statistics for variable: RH_7(t)
order 3
num_fcms 2
[ 05:33:18] getting statistics for variable: T8(t)
order 3
num_fcms 2
[ 05:33:18] getting statistics for variable: RH_8(t)
order 3
num_fcms 2
[ 05:33:18] getting statistics for variable: T9(t)
order 3
num_fcms 2
[ 05:33:18] getting statistics for variable: RH_9(t)
order 3
num_fcms 2
[ 05:33:18] getting statistics for variable: T_out(t)
order 3
num_fcms 2
[ 05:33:18] getting statistics for variable: Press_mm_hg(t)
order 3
num_fcms 2
[ 05:33:18] getting statistics for variable: RH_out(t)
order 3
num_fcms 2
[ 05:33:18] getting statistics for variable: Windspeed(t)
order 3
num_fcms 2
[ 05:33:18] getting statistics for variable: Visibility(t)
order 3
num_fcms 2
[ 05:33:18] getting statistics for variable: Tdewpoint(t)
--------------------
training window 657
shape data (164,)
forecasts1 shape (164,)
all_forecasts1 shape (1, 164)
all_forecasts2 (2, 164)
forecasts2 (164,)
order 3
num_fcms 2
[ 05:33:19] getti

C:\Users\BAHAR\anaconda3\lib\site-packages\pyFTS\benchmarks\Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
C:\Users\BAHAR\anaconda3\lib\site-packages\pyFTS\benchmarks\Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
C:\Users\BAHAR\anaconda3\lib\site-packages\pyFTS\benchmarks\Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100


 3
num_fcms 2
[ 05:33:19] getting statistics for variable: T6(t)
order 3
num_fcms 2
[ 05:33:19] getting statistics for variable: RH_6(t)
order 3
num_fcms 2
[ 05:33:19] getting statistics for variable: T7(t)
order 3
num_fcms 2
[ 05:33:19] getting statistics for variable: RH_7(t)
order 3
num_fcms 2
[ 05:33:19] getting statistics for variable: T8(t)
order 3
num_fcms 2
[ 05:33:19] getting statistics for variable: RH_8(t)
order 3
num_fcms 2
[ 05:33:19] getting statistics for variable: T9(t)
order 3
num_fcms 2
[ 05:33:19] getting statistics for variable: RH_9(t)
order 3
num_fcms 2
[ 05:33:19] getting statistics for variable: T_out(t)
order 3
num_fcms 2
[ 05:33:19] getting statistics for variable: Press_mm_hg(t)
order 3
num_fcms 2
[ 05:33:19] getting statistics for variable: RH_out(t)
order 3
num_fcms 2
[ 05:33:19] getting statistics for variable: Windspeed(t)
order 3
num_fcms 2
[ 05:33:19] getting statistics for variable: Visibility(t)
order 3
num_fcms 2
[ 05:33:19] getting statistics for va

...

C:\Users\BAHAR\anaconda3\lib\site-packages\pyFTS\benchmarks\Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100


 3
num_fcms 2
[ 05:37:26] getting statistics for variable: RH_8(t)
order 3
num_fcms 2
[ 05:37:26] getting statistics for variable: T9(t)
order 3
num_fcms 2
[ 05:37:26] getting statistics for variable: RH_9(t)
order 3
num_fcms 2
[ 05:37:26] getting statistics for variable: T_out(t)
order 3
num_fcms 2
[ 05:37:26] getting statistics for variable: Press_mm_hg(t)
order 3
num_fcms 2
[ 05:37:26] getting statistics for variable: RH_out(t)
order 3
num_fcms 2
[ 05:37:26] getting statistics for variable: Windspeed(t)
order 3
num_fcms 2
[ 05:37:26] getting statistics for variable: Visibility(t)
order 3
num_fcms 2
[ 05:37:26] getting statistics for variable: Tdewpoint(t)
--------------------
training window 18396
shape data (164,)
forecasts1 shape (164,)
all_forecasts1 shape (1, 164)
all_forecasts2 (2, 164)
forecasts2 (164,)
order 3
num_fcms 2
[ 05:37:27] getting statistics for variable: Appliances(t)
order 3
num_fcms 2
[ 05:37:27] getting statistics for variable: lights(t)
order 3
num_fcms 2
[ 05:

C:\Users\BAHAR\anaconda3\lib\site-packages\pyFTS\benchmarks\Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100


 3
num_fcms 2
[ 05:37:27] getting statistics for variable: RH_7(t)
order 3
num_fcms 2
[ 05:37:27] getting statistics for variable: T8(t)
order 3
num_fcms 2
[ 05:37:27] getting statistics for variable: RH_8(t)
order 3
num_fcms 2
[ 05:37:27] getting statistics for variable: T9(t)
order 3
num_fcms 2
[ 05:37:27] getting statistics for variable: RH_9(t)
order 3
num_fcms 2
[ 05:37:27] getting statistics for variable: T_out(t)
order 3
num_fcms 2
[ 05:37:27] getting statistics for variable: Press_mm_hg(t)
order 3
num_fcms 2
[ 05:37:27] getting statistics for variable: RH_out(t)
order 3
num_fcms 2
[ 05:37:27] getting statistics for variable: Windspeed(t)
order 3
num_fcms 2
[ 05:37:27] getting statistics for variable: Visibility(t)
order 3
num_fcms 2
[ 05:37:27] getting statistics for variable: Tdewpoint(t)
--------------------
training window 19053
shape data (164,)
forecasts1 shape (164,)
all_forecasts1 shape (1, 164)
all_forecasts2 (2, 164)
forecasts2 (164,)
order 3
num_fcms 2
[ 05:37:28] get

C:\Users\BAHAR\anaconda3\lib\site-packages\pyFTS\benchmarks\Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100


### Final Results

In [ ]:
import statistics
#ttest = df.loc[:,:]

columns = list(reframed.loc[:,'Appliances(t)':'Tdewpoint(t)'].columns)

final_result = {
    "variable": [],
     "rmse": [],
     "mae": [],
     "r2": [],
     "nrmse": [],
    "mape": []
}

measures = kpca_result
var = measures.groupby("variable")

for col in columns:

    var_agr = var.get_group(col)

    rmse = round(statistics.mean(var_agr.loc[:,'rmse']),3)
    nrmse = round(statistics.mean(var_agr.loc[:,'nrmse']),3)
    mae = round(statistics.mean(var_agr.loc[:,'mae']),3)
    r2 = round(statistics.mean(var_agr.loc[:,'r2']),3)
    mape = round(statistics.mean(var_agr.loc[:,'mape']),3)

    final_result["variable"].append(col)
    final_result["rmse"].append(rmse)
    final_result["nrmse"].append(nrmse)
    final_result["mae"].append(mae)
    final_result["r2"].append(r2)
    final_result["mape"].append(mape)


    print(f'Results: {(col,rmse,mae,r2,nrmse,mape)}')


final_measures_kpca = pd.DataFrame(final_result)

print("Statistics MIMO (test): ")
final_measures_kpca

Results: ('Appliances(t)', 66.826, 34.867, 0.38, 0.128, 35.909)
Results: ('lights(t)', 5.322, 3.174, 0.142, nan, nan)
Results: ('T1(t)', 0.202, 0.157, 0.839, 0.101, 0.737)
Results: ('RH_1(t)', 0.574, 0.362, 0.851, 0.07, 0.886)
Results: ('T2(t)', 0.425, 0.303, 0.82, 0.11, 1.501)
Results: ('RH_2(t)', 0.707, 0.491, 0.86, 0.086, 1.223)
Results: ('T3(t)', 0.121, 0.092, 0.904, 0.07, 0.411)
Results: ('RH_3(t)', 0.334, 0.225, 0.866, 0.079, 0.567)
Results: ('T4(t)', 0.198, 0.148, 0.812, 0.099, 0.716)
Results: ('RH_4(t)', 0.333, 0.24, 0.9, 0.072, 0.618)
Results: ('T5(t)', 0.15, 0.096, 0.866, 0.079, 0.495)
Results: ('RH_5(t)', 2.151, 1.02, 0.84, 0.079, 1.872)
Results: ('T6(t)', 1.201, 0.84, 0.775, 0.128, inf)
Results: ('RH_6(t)', 4.342, 3.007, 0.84, nan, 35.412)
Results: ('T7(t)', 0.117, 0.094, 0.884, 0.085, 0.478)
Results: ('RH_7(t)', 0.734, 0.536, 0.804, 0.11, 1.535)
Results: ('T8(t)', 0.17, 0.125, 0.894, 0.082, 0.583)
Results: ('RH_8(t)', 0.937, 0.651, 0.836, 0.109, 1.53)
Results: ('T9(t)', 0.

,variable,rmse,mae,r2,nrmse,mape
0,Appliances(t),66.826,34.867,0.380,0.128,35.909
1,lights(t),5.322,3.174,0.142,NaN,NaN
2,T1(t),0.202,0.157,0.839,0.101,0.737
3,RH_1(t),0.574,0.362,0.851,0.070,0.886
4,T2(t),0.425,0.303,0.820,0.110,1.501
5,RH_2(t),0.707,0.491,0.860,0.086,1.223
6,T3(t),0.121,0.092,0.904,0.070,0.411
7,RH_3(t),0.334,0.225,0.866,0.079,0.567
8,T4(t),0.198,0.148,0.812,0.099,0.716
9,RH_4(t),0.333,0.240,0.900,0.072,0.618


### Sensitivity Analysis

for each window:
  train the model
  for n in range(10):
    model.layer.weights += np.random.randn(0, 10^-n)

In [ ]:
from copy import deepcopy

data = reframed
Deep_random_HFCM=DRHFCM_PCA(num_components_pca =2,gamma = 0.9)
n_windows=30
train_size=0.75
transformation='kPCA'

result = {
      "window": [],
      "noise":[],
      "rmse": [],
      "mape": [],
      "mae": [],
      "r2": [],
      "nrmse": [],
      "variable":[]
}

final_result = {
      "window": [],
      "noise":[],
      "rmse": [],
      "mape": [],
      "mae": [],
      "r2": [],
      "nrmse": [],
      "variable":[]
}

tam = len(data)
n_windows = 30
windows_length = math.floor(tam / n_windows)
for ct, ttrain, ttest in Util.sliding_window(data, windows_length, train_size, inc=1):
    if len(ttest) > 0:

        print('-' * 20)
        print(f'training window {(ct)}')


        df_train = ttrain.loc[:,'Appliances(t-1)':'Tdewpoint(t-1)']
        df_test = ttest.loc[:,'Appliances(t-1)':'Tdewpoint(t-1)']
        df_original = ttest.loc[:,'Appliances(t)':'Tdewpoint(t)']

        models, data_train = Deep_random_HFCM.run_train(df_train, transformation)

        for i in range(6,-1,-1):

          variance = 10**(-i)

          new_models = deepcopy(models)

          for model in new_models:
            for fcm in model.fcm:
              num = len(fcm.weights)
              for k in range(num):
                shape1 = fcm.weights[k].shape
                noise1 = np.random.randn(*shape1)*variance
                fcm.weights[k] += noise1

                shape2 = fcm.bias[k].shape
                noise2 = np.random.randn(*shape2)*variance
                fcm.bias[k] += noise2

          forecast, data_test = Deep_random_HFCM.run_test(new_models,df_test,transformation)


          df_forecats_columns = ['Appliances(t)', 'lights(t)', 'T1(t)', 'RH_1(t)', 'T2(t)', 'RH_2(t)',
                                  'T3(t)', 'RH_3(t)', 'T4(t)', 'RH_4(t)', 'T5(t)', 'RH_5(t)', 'T6(t)',
                                  'RH_6(t)', 'T7(t)', 'RH_7(t)', 'T8(t)', 'RH_8(t)', 'T9(t)', 'RH_9(t)',
                                  'T_out(t)', 'Press_mm_hg(t)', 'RH_out(t)', 'Windspeed(t)',
                                  'Visibility(t)', 'Tdewpoint(t)']
          columns = list(df_forecats_columns)
          df_forecast = pd.DataFrame(forecast,columns=columns)

          for col in columns:
            original = df_original[col].values
            forecast = df_forecast[col].values

            print("[{0: %H:%M:%S}]".format(datetime.datetime.now()) + f" getting statistics for variable: " + col)

            mae = round(mean_absolute_error(original,forecast),3)
            r2 = round(r2_score(original,forecast),3)
            #rmse = mean_squared_error(original,forecast,squared=False)
            rmse = round(Measures.rmse(original,forecast),3)
            mape = round(Measures.mape(original,forecast),3)
            nrmse = round(cal_nrmse(rmse, original),3)

            result["rmse"].append(rmse)
            result["nrmse"].append(nrmse)
            result["mape"].append(mape)
            result["mae"].append(mae)
            result["r2"].append(r2)
            result["window"].append(ct)
            result["noise"].append(variance)
            result["variable"].append(col)

measures = pd.DataFrame(result)

--------------------
training window 0
[ 13:15:49] getting statistics for variable: Appliances(t)
[ 13:15:49] getting statistics for variable: lights(t)
[ 13:15:49] getting statistics for variable: T1(t)
[ 13:15:49] getting statistics for variable: RH_1(t)
[ 13:15:49] getting statistics for variable: T2(t)
[ 13:15:49] getting statistics for variable: RH_2(t)
[ 13:15:49] getting statistics for variable: T3(t)
[ 13:15:49] getting statistics for variable: RH_3(t)
[ 13:15:49] getting statistics for variable: T4(t)
[ 13:15:49] getting statistics for variable: RH_4(t)
[ 13:15:49] getting statistics for variable: T5(t)
[ 13:15:49] getting statistics for variable: RH_5(t)
[ 13:15:49] getting statistics for variable: T6(t)
[ 13:15:49] getting statistics for variable: RH_6(t)
[ 13:15:49] getting statistics for variable: T7(t)
[ 13:15:49] getting statistics for variable: RH_7(t)
[ 13:15:49] getting statistics for variable: T8(t)
[ 13:15:49] getting statistics for variable: RH_8(t)
[ 13:15:49] get

/usr/local/lib/python3.10/dist-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/usr/local/lib/python3.10/dist-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100


[ 13:15:49] getting statistics for variable: Appliances(t)
[ 13:15:49] getting statistics for variable: lights(t)
[ 13:15:49] getting statistics for variable: T1(t)
[ 13:15:49] getting statistics for variable: RH_1(t)
[ 13:15:49] getting statistics for variable: T2(t)
[ 13:15:49] getting statistics for variable: RH_2(t)
[ 13:15:49] getting statistics for variable: T3(t)
[ 13:15:49] getting statistics for variable: RH_3(t)
[ 13:15:49] getting statistics for variable: T4(t)
[ 13:15:49] getting statistics for variable: RH_4(t)
[ 13:15:49] getting statistics for variable: T5(t)
[ 13:15:49] getting statistics for variable: RH_5(t)
[ 13:15:49] getting statistics for variable: T6(t)
[ 13:15:49] getting statistics for variable: RH_6(t)
[ 13:15:49] getting statistics for variable: T7(t)
[ 13:15:49] getting statistics for variable: RH_7(t)
[ 13:15:49] getting statistics for variable: T8(t)
[ 13:15:49] getting statistics for variable: RH_8(t)
[ 13:15:49] getting statistics for variable: T9(t)
[ 1

/usr/local/lib/python3.10/dist-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/usr/local/lib/python3.10/dist-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100


[ 13:15:49] getting statistics for variable: Appliances(t)
[ 13:15:49] getting statistics for variable: lights(t)
[ 13:15:49] getting statistics for variable: T1(t)
[ 13:15:49] getting statistics for variable: RH_1(t)
[ 13:15:49] getting statistics for variable: T2(t)
[ 13:15:49] getting statistics for variable: RH_2(t)
[ 13:15:50] getting statistics for variable: T3(t)
[ 13:15:50] getting statistics for variable: RH_3(t)
[ 13:15:50] getting statistics for variable: T4(t)
[ 13:15:50] getting statistics for variable: RH_4(t)
[ 13:15:50] getting statistics for variable: T5(t)
[ 13:15:50] getting statistics for variable: RH_5(t)
[ 13:15:50] getting statistics for variable: T6(t)
[ 13:15:50] getting statistics for variable: RH_6(t)
[ 13:15:50] getting statistics for variable: T7(t)
[ 13:15:50] getting statistics for variable: RH_7(t)
[ 13:15:50] getting statistics for variable: T8(t)
[ 13:15:50] getting statistics for variable: RH_8(t)
[ 13:15:50] getting statistics for variable: T9(t)
[ 1

...

[ 13:17:54] getting statistics for variable: Appliances(t)
[ 13:17:54] getting statistics for variable: lights(t)
[ 13:17:54] getting statistics for variable: T1(t)
[ 13:17:54] getting statistics for variable: RH_1(t)
[ 13:17:54] getting statistics for variable: T2(t)
[ 13:17:54] getting statistics for variable: RH_2(t)
[ 13:17:54] getting statistics for variable: T3(t)
[ 13:17:54] getting statistics for variable: RH_3(t)
[ 13:17:54] getting statistics for variable: T4(t)
[ 13:17:54] getting statistics for variable: RH_4(t)
[ 13:17:54] getting statistics for variable: T5(t)
[ 13:17:54] getting statistics for variable: RH_5(t)
[ 13:17:54] getting statistics for variable: T6(t)
[ 13:17:54] getting statistics for variable: RH_6(t)
[ 13:17:54] getting statistics for variable: T7(t)
[ 13:17:54] getting statistics for variable: RH_7(t)
[ 13:17:54] getting statistics for variable: T8(t)
[ 13:17:54] getting statistics for variable: RH_8(t)
[ 13:17:54] getting statistics for variable: T9(t)
[ 1

/usr/local/lib/python3.10/dist-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100


[ 13:17:55] getting statistics for variable: Appliances(t)
[ 13:17:55] getting statistics for variable: lights(t)
[ 13:17:55] getting statistics for variable: T1(t)
[ 13:17:55] getting statistics for variable: RH_1(t)
[ 13:17:55] getting statistics for variable: T2(t)
[ 13:17:55] getting statistics for variable: RH_2(t)
[ 13:17:55] getting statistics for variable: T3(t)
[ 13:17:55] getting statistics for variable: RH_3(t)
[ 13:17:55] getting statistics for variable: T4(t)
[ 13:17:55] getting statistics for variable: RH_4(t)
[ 13:17:55] getting statistics for variable: T5(t)
[ 13:17:55] getting statistics for variable: RH_5(t)
[ 13:17:55] getting statistics for variable: T6(t)
[ 13:17:55] getting statistics for variable: RH_6(t)
[ 13:17:55] getting statistics for variable: T7(t)
[ 13:17:55] getting statistics for variable: RH_7(t)
[ 13:17:55] getting statistics for variable: T8(t)
[ 13:17:55] getting statistics for variable: RH_8(t)
[ 13:17:55] getting statistics for variable: T9(t)
[ 1

/usr/local/lib/python3.10/dist-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100


SyntaxError: 'return' outside function (<ipython-input-43-837a6836705c>, line 99)

In [ ]:
measures

,window,noise,rmse,mape,mae,r2,nrmse,variable
0,0,0.000001,114.423,109.919,88.734,0.052,0.238,Appliances(t)
1,0,0.000001,6.872,inf,5.836,0.066,0.229,lights(t)
2,0,0.000001,0.726,2.752,0.606,0.043,0.290,T1(t)
3,0,0.000001,1.872,3.002,1.178,0.003,0.136,RH_1(t)
4,0,0.000001,0.755,2.870,0.608,0.047,0.225,T2(t)
...,...,...,...,...,...,...,...,...
5455,19053,1.000000,0.834,0.094,0.714,-2.358,0.463,Press_mm_hg(t)
5456,19053,1.000000,30.533,46.307,26.848,-3.179,0.679,RH_out(t)
5457,19053,1.000000,0.929,44.477,0.787,-0.140,0.310,Windspeed(t)
5458,19053,1.000000,16.894,42.189,14.046,-0.378,0.384,Visibility(t)


In [ ]:
import statistics
#ttest = df.loc[:,:]

columns = list(reframed.loc[:,'Appliances(t)':'Tdewpoint(t)'].columns)

noises = measures["noise"].unique()

final_result = {
    "noise": [],
    "variable": [],
     "rmse": [],
     "mae": [],
     "r2": [],
     "nrmse": [],
    "mape": []
}

var = measures.groupby("variable")

for col in columns:

  var_agr1 = var.get_group(col)

  var2 = var_agr1.groupby("noise")

  for noise in noises:

    var_agr = var2.get_group(noise)

    rmse = round(statistics.mean(var_agr.loc[:,'rmse']),3)
    nrmse = round(statistics.mean(var_agr.loc[:,'nrmse']),3)
    mae = round(statistics.mean(var_agr.loc[:,'mae']),3)
    r2 = round(statistics.mean(var_agr.loc[:,'r2']),3)
    mape = round(statistics.mean(var_agr.loc[:,'mape']),3)

    final_result["noise"].append(noise)
    final_result["variable"].append(col)
    final_result["rmse"].append(rmse)
    final_result["nrmse"].append(nrmse)
    final_result["mae"].append(mae)
    final_result["r2"].append(r2)
    final_result["mape"].append(mape)


    print(f'Results: {(noise,col,rmse,mae,r2,nrmse,mape)}')


final_measures_kpca = pd.DataFrame(final_result)

print("Statistics MIMO (test): ")
final_measures_kpca

Results: (1e-06, 'Appliances(t)', 88.341, 56.588, -0.024, 0.17, 67.425)
Results: (1e-05, 'Appliances(t)', 88.341, 56.588, -0.024, 0.17, 67.425)
Results: (0.0001, 'Appliances(t)', 88.342, 56.588, -0.024, 0.17, 67.425)
Results: (0.001, 'Appliances(t)', 88.356, 56.581, -0.024, 0.17, 67.399)
Results: (0.01, 'Appliances(t)', 88.387, 56.378, -0.022, 0.169, 66.844)
Results: (0.1, 'Appliances(t)', 94.009, 54.358, -0.109, 0.176, 51.512)
Results: (1.0, 'Appliances(t)', 95.49, 54.952, -0.161, 0.18, 50.932)
Results: (1e-06, 'lights(t)', 6.88, 5.154, 0.023, nan, nan)
Results: (1e-05, 'lights(t)', 6.88, 5.154, 0.023, nan, nan)
Results: (0.0001, 'lights(t)', 6.88, 5.154, 0.023, nan, nan)
Results: (0.001, 'lights(t)', 6.88, 5.154, 0.023, nan, nan)
Results: (0.01, 'lights(t)', 6.902, 5.127, 0.018, nan, nan)
Results: (0.1, 'lights(t)', 7.434, 4.895, -0.094, nan, nan)
Results: (1.0, 'lights(t)', 7.64, 5.02, -0.169, nan, nan)
Results: (1e-06, 'T1(t)', 0.656, 0.543, -0.123, 0.317, 2.531)
Results: (1e-05, '

,noise,variable,rmse,mae,r2,nrmse,mape
0,0.000001,Appliances(t),88.341,56.588,-0.024,0.170,67.425
1,0.000010,Appliances(t),88.341,56.588,-0.024,0.170,67.425
2,0.000100,Appliances(t),88.342,56.588,-0.024,0.170,67.425
3,0.001000,Appliances(t),88.356,56.581,-0.024,0.170,67.399
4,0.010000,Appliances(t),88.387,56.378,-0.022,0.169,66.844
...,...,...,...,...,...,...,...
177,0.000100,Tdewpoint(t),1.461,1.170,-0.166,0.282,inf
178,0.001000,Tdewpoint(t),1.460,1.171,-0.166,0.282,inf
179,0.010000,Tdewpoint(t),1.468,1.172,-0.173,0.283,inf
180,0.100000,Tdewpoint(t),1.536,1.241,-0.367,0.301,inf


In [ ]:
measures["noise"].unique()

array([1.e-06, 1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00])

In [ ]:
final_measures_kpca.to_csv("sensitivity_analysis.csv", sep=",", index=False)